## Library Imports

In [30]:
import numpy as np  
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Better visualisation
import pandas as pd  
import datetime  # Date and time operations
from datetime import datetime 
from sklearn.metrics import mean_squared_error  
from sklearn.preprocessing import StandardScaler  # Feature scaling
from keras.models import load_model  
from binance.client import Client  # Binance API interaction
from binance.exceptions import BinanceAPIException, BinanceOrderException  # API exceptions
import time  

# Rate limiter setup
def rate_limiter(max_per_second):
    """
    Decorator function to limit the rate of API calls.

    Args:
        max_per_second (int): Maximum number of API calls allowed per second.

    Returns:
        function: Decorated function with rate limiting.
    """
    min_interval = 1.0 / float(max_per_second)
    def decorate(func):
        last_called = [0.0]
        def rate_limited_function(*args, **kargs):
            elapsed = time.time() - last_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            last_called[0] = time.time()
            return func(*args, **kargs)
        return rate_limited_function
    return decorate

The framework for data analysis and visualisation is built up by this code, which also includes libraries for managing time-related activities, graphing and numerical computation. It also provides a rate limiter function that controls the number of times an API call is made while using the Binance API.

## Binance API Setup

In [31]:
# Define Binance API keys
api_key = ""    # Binance API key
secret_key = ""  # Binance API secret key

# Binance client with error handling
try:
    client = Client(api_key, secret_key)
    print("Client Loaded Successfully")
except Exception as e:
    print(f"Error initializing Binance Client: {e}")

Client Loaded Successfully


## Fetch the Data 

In [32]:
# Fetch candlestick data from Binance with error handling
symbol = 'BTCUSDT'  # Symbol for Bitcoin against USDT
interval = Client.KLINE_INTERVAL_15MINUTE  # Interval for candlestick data
limit = 1000  # Number of data points to fetch

try:
    dataset = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    print("Data fetch successful")
except BinanceAPIException as e:
    print(f"API Exception occurred: {e.status_code} - {e.message}")
except BinanceOrderException as e:
    print(f"Order Exception occurred: {e.status_code} - {e.message}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data fetch successful


## Preprocessing the Data

In [33]:
# Preprocess the data
price = np.array([float(dataset[i][4]) for i in range(len(dataset))])
price = price.reshape(-1, 1) 

# StandardScaler
scaler = StandardScaler()
scaler.fit(price[:800])  # Fit the scaler on the first 800 candles for training
prices_scaled = scaler.transform(price)  # Transform the entire dataset

StandardScaler is used to standardise the data in order to guarantee consistency and get the data ready for live trading.

## Loading the Model

In [34]:
# Load LR model
import joblib

# Load the trained Linear Regression model
loaded_model = joblib.load("Final_LR.pkl")

# Inspect the model's attributes and coefficients
print("Intercept:", loaded_model.intercept_)
print("Coefficients:", loaded_model.coef_)


Intercept: 0.012171757406324195
Coefficients: [-0.06273781  0.01540444  0.14570403  0.89632762]


In [35]:
import warnings

# Suppressing the warning
warnings.filterwarnings('ignore', message="X does not have valid feature names")

In [36]:
index = [996, 997, 998, 999]
candles = scaler.transform(np.array([float(dataset[i][4]) for i in index]).reshape(-1,1))

model_feed = candles.reshape(1, 4)  # reshape to (1, 4)

prediction = loaded_model.predict(model_feed)

# We reshape it to (-1, 1) to make it a 2D array with a single column
prediction_reshaped = prediction.reshape(-1, 1)

# Now we can  apply the inverse transform
predicted_price = scaler.inverse_transform(prediction_reshaped)[0][0]

print(predicted_price)

67591.97855020862


A loaded model is used  to estimate the future price after reshaping the scaled candle data to the appropriate format. To get the predicted cost in the original scale, it then performs the inverse transform.

## Risk Management

In [37]:
# Define stop loss and take profit percentages
stop_loss_percentage = 0.05  # 5%
take_profit_percentage = 0.1  # 10%

account_balance = 20  # Based on my actual account balance
risk_per_trade = 0.02

In [38]:
# Define a function to calculate the position size based on account balance, risk per trade and stop loss percentage
def calculate_position_size(account_balance, risk_per_trade, stop_loss_percentage):
    """
    Calculates the position size based on the account balance, risk per trade and stop loss percentage.

    """
    risk_amount = account_balance * risk_per_trade
    position_size = risk_amount / (stop_loss_percentage * account_balance)
    return position_size

# Calculate the position size
position_size = calculate_position_size(account_balance, risk_per_trade, stop_loss_percentage)

# Print the calculated position size
print(f"Position size based on account balance and risk per trade: {position_size}")

Position size based on account balance and risk per trade: 0.4


In [39]:
@rate_limiter(1)  # Apply the rate limiter to control API calls
def get_current_price(symbol):
    try:
        current_prices = client.get_symbol_ticker(symbol=symbol)
        current_price = float(current_prices['price'])
        return current_price
    except BinanceAPIException as e:
        print(f"API Exception occurred: {e.status_code} - {e.message}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

The present value of a specified cryptocurrency symbol (BTC) can be obtained by this function from the Binance API and a rate limitations is implemented to regulate the frequency of API requests. It correctly manages unforeseen problems and Binance API exceptions.

# Prediction Example

In [40]:
# Example Prediction and Trading Logic
current_price = get_current_price(symbol)

# Predict next price using the loaded model
latest_prices_scaled = prices_scaled[-4:].reshape(1, 4)

# Predict next price using the loaded model
predicted_price_scaled = loaded_model.predict(latest_prices_scaled)

# Inverse transform the predicted price to get the actual price
predicted_price = scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1))[0][0]

# Print the current and predicted prices
print(f"Current Price: {current_price}, Predicted Price: {predicted_price}")

Current Price: 67576.56, Predicted Price: 67591.97855020862


# Tarding Logic And Actual Trading

In [ ]:
# Trading Logic And Actual Trading
symbol = "BTCUSDT"  # Symbol of Pair
quantity = '0.05'  # Quantity to trade
order = False

while True:
    try:
        # Fetch recent trades for the current price
        recent_trades = client.get_recent_trades(symbol=symbol)
        current_price = float(recent_trades[-1]['price'])  

        # Fetch candle data for prediction
        candles = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE, limit=4)
        model_data = np.array([float(candle[4]) for candle in candles]).reshape(-1, 1)
        model_data_scaled = scaler.transform(model_data).reshape(1, 4)

        # Predict the next price
        predicted_price_scaled = loaded_model.predict(model_data_scaled)
        predicted_price = scaler.inverse_transform(predicted_price_scaled.reshape(-1, 1))[0][0]

        print(f"Current Price: {current_price}, Predicted Price: {predicted_price}")

        # Trading decision based on the prediction
        if not order and predicted_price > current_price:
            order = True
            # Place a market buy order
            #buy_price = client.get_order_book(symbol=symbol)['asks'][0][0]
            #buy_order = client.order_market_buy(symbol=symbol, quantity=quantity)
            print(f"Buy @Market Price: {current_price}, Timestamp: {datetime.now()}")

        elif order and current_price > predicted_price:
            order = False
            # Place a market sell order
            #sell_price = client.get_order_book(symbol=symbol)['bids'][0][0]
            #sell_order = client.order_market_sell(symbol=symbol, quantity=quantity)
            print(f"Sell @Market Price: {current_price}, Timestamp: {datetime.now()}")

    except BinanceAPIException as e:
        print(f"Binance API Exception: {e.message}")
    except BinanceOrderException as e:
        print(f"Binance Order Exception: {e.message}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    time.sleep(60)  # Wait before the next iteration to avoid hitting rate limits

Current Price: 67576.56, Predicted Price: 67591.96958693239
Buy @Market Price: 67576.56, Timestamp: 2024-04-13 14:57:57.527157
Current Price: 67542.77, Predicted Price: 67561.69163985024
Current Price: 67552.02, Predicted Price: 67569.97370708053
Current Price: 67588.02, Predicted Price: 67602.45544869806
Current Price: 67639.24, Predicted Price: 67648.36534951357
Current Price: 67638.23, Predicted Price: 67647.460058615
Current Price: 67510.0, Predicted Price: 67532.52396760424
Current Price: 67512.0, Predicted Price: 67534.31662284888
Current Price: 67547.99, Predicted Price: 67566.57545397637
Current Price: 67599.1, Predicted Price: 67612.38675875343
Current Price: 67642.48, Predicted Price: 67651.26048773367
Current Price: 67635.6, Predicted Price: 67645.10271696831
Current Price: 67609.99, Predicted Price: 67622.15672983677
Current Price: 67622.01, Predicted Price: 67632.93058785712
Current Price: 67608.0, Predicted Price: 67620.37303786834
Current Price: 67634.37, Predicted Price

A basic trading strategy based on price forecasts is demonstrated in this code section. It runs in a loop, regularly retrieving information from candlesticks and recent transaction data to predict the next change in price. It predicts the direction of price movement and then decides whether to execute a market buy or sell order for BTC Price. A comparison between the predicted cost and the current market price helps the process of decision-making. All noticed errors are appropriately handled to guarantee seamless operation, especially order exceptions and API problems. The script stops briefly in between iterations to avoid going above rate limitations.

## References


- Project on the Development of a Crypto Trading Bot. Detailed guide on creating and managing a cryptocurrency trading bot using Python and Binance API. Available at: https://dev.to/nicolasbonnici/how-to-build-a-crypto-bot-with-python-3-and-the-binance-api-part-1-1864
- Application of Technical Analysis Strategies in Cryptocurrency Trading. Explanation and implementation of technical trading strategies using the Binance API. Available at: https://www.learndatasci.com/tutorials/algo-trading-crypto-bot-python-strategy-backtesting/

- Handling Errors and API Management in Crypto Trading. Discussion on robust error handling techniques for maintaining operational stability of trading bots on Binance. Available at: https://www.learndatasci.com/tutorials/algo-trading-crypto-bot-python-strategy-backtesting/

- Binance Trading API – Creating Your First Simple Crypto-Trading Bot. Available at: https://blog.finxter.com/binance-api-trading-bot/

- Binance API documentation. https://binance-docs.github.io/apidocs/spot/en/

- Al-Nefaie, A.H. and Aldhyani, T.H.H., 2022. Bitcoin Price Forecasting and Trading: Data Analytics Approaches. Electronics, 11(24), p.4088. Available at: https://www.mdpi.com/2079-9292/11/24/4088.